# Подзапросы, представления, создание таблиц

Используем подзапросы и представления таблиц, вставляем новые данные. Создаем, сохраняем и удаляем таблицы.

**Данные: Airbnb в Берлине.**

Имеются следующие таблицы:

listings – информация о жилье, включая полные описания, характеристики и средние оценки в отзывах; поскольку столбцов очень много, нужные перечислены в текстах самих задач

calendar_summary – информация о доступности и цене того или иного жилья по дням

- listing_id – идентификатор жилья (объявления)
- date – дата
- available – доступность жилья в данный день (t/f)
- price – цена за ночь

reviews – отзывы

- listing_id –  идентификатор жилья  
- id – id отзыва
- date – дата отзыва
- reviewer_id – id ревьюера (автора отзыва)
- reviewer_name – имя автора
- comments – сам отзыв



1) Сначала оставьте только те объявления, в которых оценка на основе отзывов выше среднего, а число отзывов в месяц составляет строго меньше трёх. Затем отсортируйте по убыванию две колонки: сначала по числу отзывов в месяц, потом по оценке. В качестве ответа укажите id объявления из первой строки.

    SELECT
        id,
        toFloat64OrNull(review_scores_rating),
        reviews_per_month
    FROM
        listings 
    WHERE
        toFloat64OrNull(review_scores_rating) > (SELECT 
                                            AVG(toFloat64OrNull(review_scores_rating)) 
                    FROM listings)
            AND reviews_per_month < 3
    ORDER BY
        reviews_per_month DESC,
        review_scores_rating DESC 
    LIMIT 10



2) Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем. 

В качестве ответа укажите идентификатор хозяина (host_id), сдающего данную комнату.

    SELECT
        host_id, 
        geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance,
        room_type

    FROM
        listings 
    WHERE
             distance < (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) FROM 
            (SELECT * FROM listings WHERE room_type = 'Private room'))
            AND room_type = 'Private room'

    ORDER BY distance DESC
    LIMIT 1

3) Представим, что вы планируете снять жилье в Берлине на 7 дней, используя более хитрые фильтры, чем предлагаются на сайте.
В этой задаче можно потренироваться в написании подзапросов, но задание можно решить и без них.

Отберите объявления из таблицы listings, которые:
- находятся на расстоянии от центра меньше среднего (здесь нам пригодится запрос из предыдущего задания)
- обойдутся дешевле 100 долларов в день (price с учетом cleaning_fee, который добавляется к общей сумме за неделю, т.е его нужно делить на кол-во дней)
- имеют последние отзывы (last_review), начиная с 1 сентября 2018 года
- имеют WiFi в списке удобств (amenities)

Отсортируйте полученные значения по убыванию review_scores_rating (не забудьте перевести строку к численному виду) и в качестве ответа укажите host_id из первой строки. 


    SELECT
        host_id, 
        geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance,
        toFloat64OrZero(replaceRegexpAll(price, '[$,]', '')) AS price,
    (toFloat32OrZero(replaceRegexpAll(cleaning_fee, '[$,]', '')))/7 AS cleaning_fee,
        price + cleaning_fee AS total_price,
        toFloat64OrNull(review_scores_rating) AS review_scores_rating,
        toDateOrNull(last_review) AS last_review,
        amenities

    FROM
        listings 
    WHERE
            distance < (SELECT AVG(geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude))) FROM listings)
        AND total_price < 100
        AND last_review > '2018-09-01'
        AND multiSearchAnyCaseInsensitive(amenities, ['WiFi']) != 0  

    ORDER BY review_scores_rating DESC
    LIMIT 1



**Данные: мобильное приложение.**

installs — содержит данные об установках приложения по дням.

- DeviceID — идентификатор устройства, на которое было установлено приложение;
- InstallationDate — дата установки приложения;
- InstallCost — цена установки приложения в рублях;
- Platform — платформа, на которой было установлено приложение (iOS/ Android);
- Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).

events — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

- DeviceID — идентификатор устройства, на котором используется приложение;
- AppPlatform — платформа, на которой используется приложение (iOS/ Android);
- EventDate — дата, за которую собрана статистика;
- events — количество просмотров всех товаров за этот день у этого DeviceID.

checks — содержит данные о покупках пользователей в приложении по дням

- UserID — идентификатор пользователя;
- Rub — суммарный чек пользователя на дату;
- BuyDate — дата, за которую собрана статистика.

devices – чтобы просмотры и установки можно было объединить с покупками
 
- DeviceID — идентификатор устройства;
- UserID — идентификатор пользователя.


С помощью оператора CASE в SQL можно составлять условные конструкции, выполнять проверку условий и возвращать результат в зависимости от их выполнения. Самих условий под оператором WHEN может быть сколько угодно, конструкцию необходимо завершить оператором END. 

4) Используйте таблицу checks и разделите всех покупателей на сегменты:

- А — средний чек покупателя менее 5 ₽
- B — средний чек покупателя от 5-10 ₽
- C — средний чек покупателя от 10-20 ₽
- D — средний чек покупателя от 20 ₽

Отсортируйте результирующую таблицу по возрастанию UserID и укажите сегмент четвертого пользователя.


    SELECT 
        CASE
            WHEN AVG(Rub) < 5 THEN 'A'
            WHEN (AVG(Rub) = 5 OR AVG(Rub) > 5) AND AVG(Rub) < 10 THEN 'B'
            WHEN (AVG(Rub) = 10 OR AVG(Rub) > 10) AND AVG(Rub) < 20 THEN 'C'
            ELSE 'D'
        END AS cond, -- END – завершаем конструкцию, а новый столбец называем cond_1
        AVG(Rub),
        UserID  
    FROM
        checks
    GROUP BY
        UserID
    ORDER BY
        UserID ASC  
    LIMIT 4


5) Используйте предыдущий запрос как подзапрос и посчитайте, сколько клиентов приходится на каждый сегмент и сколько доходов он приносит. Отсортируйте результат по убыванию суммы доходов на сегмент и в качестве ответа укажите наибольшую сумму.

    SELECT 
        count(l.UserID),
        l.cond,
        sum(r.Rub) as sum_rub

    FROM (
        SELECT 
            CASE
                WHEN AVG(Rub) < 5 THEN 'A'
                WHEN (AVG(Rub) = 5 OR AVG(Rub) > 5) AND AVG(Rub) < 10 THEN 'B'
                WHEN (AVG(Rub) = 10 OR AVG(Rub) > 10) AND AVG(Rub) < 20 THEN 'C'
                ELSE 'D'
            END AS cond,
            UserID
        FROM checks
        GROUP BY UserID
        ) as l

    JOIN (
        SELECT 
            UserID, 
            SUM(Rub) as Rub  
        FROM checks 
        GROUP BY UserID
        ) AS r

        ON l.UserID = r.UserID 

    GROUP BY l.cond
    ORDER BY sum_rub DESC  


6) Вернемся к таблице **Airbnb**. Предположим, что в выборе жилья нас интересует только два параметра: наличие кухни (kitchen) и гибкой системы отмены (flexible), причем первый в приоритете.

Создайте с помощью оператора CASE колонку с обозначением группы, в которую попадает жилье из таблицы listings:

- 'good', если в удобствах (amenities) присутствует кухня и система отмены (cancellation_policy) гибкая
- 'ok', если в удобствах есть кухня, но система отмены не гибкая
- 'not ok' во всех остальных случаях

Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк, в качестве ответа укажите host_id первой строки.
Обратите внимание, что cancellation_policy - это отдельная колонка, по ней необходимо смотреть систему отмены


    SELECT 
        host_id,
        CASE
            WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) != 0 AND multiSearchAnyCaseInsensitive(cancellation_policy, ['flexible']) != 0 THEN 'good'
            WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen']) != 0 AND multiSearchAnyCaseInsensitive(cancellation_policy, ['flexible']) == 0 THEN 'ok'
            ELSE 'not ok'
        END AS cond
    FROM listings 
    ORDER BY cond ASC
    LIMIT 5


7) Напишите запрос для создания таблицы со следующими параметрами, также подобрав подходящий тип данных.

Название таблицы: reviews

База данных: test

Столбцы:
- listing_id – идентификатор объявления, может быть только положительным и целым числом, 32-битный тип данных
- id – идентификатор хозяина, может быть только положительным и целым числом, 32-битный тип данных
- created_at – дата со временем (2020-01-01 00:00:00), часовой пояс – 'Europe/Moscow'
- reviewer_id – идентификатор ревьюера, может быть только положительным и целым числом, 32-битный тип данных
- reviewer_name – имя того, кто оставил отзыв
- comments - текст отзыва

Движок: MergeTree

Сортировка: listing_id, id

    CREATE TABLE test.reviews (
        listing_id UInt32, -- колонка, тип данных
        id UInt32,
        created_at DateTime('Europe/Moscow'),
        reviewer_id UInt32,
        reviewer_name String,
        comments String  
        ) 

    ENGINE = MergeTree -- движок
    ORDER BY (listing_id, id)




8) К вам пришел коллега с новостями: оказывается, в поле created_at будет записываться только дата, без времени, поэтому нужно изменить тип данных!

Напишите запрос, который модифицирует тип данных, и введите его в поле ниже без кавычек и лишних пробелов. Не забудьте указать базу данных test перед названием таблицы! 


    ALTER TABLE test.reviews MODIFY COLUMN created_at Date

9) Предположим, ваш коллега вставил данные, но что-то перепутал. Часть строк с комментариями осталась совершенно пустой! 
Напишите запрос, который удалит из таблицы test.reviews те строки, где в столбце comments встречаются пустые значения (''). Введите его в поле ниже без кавычек и лишних пробелов.

    ALTER TABLE test.reviews DELETE WHERE comments = ''

10) Напишите запрос, который удвоит price для всех строк с датой (created_at) после 2019-01-01.

    ALTER TABLE test.reviews UPDATE price = price * 2 WHERE created_at > '2019-01-01'
